In [2]:
import pandas as pd
import csv

# 1. 读取 simms-eu.csv
src_file = r".\p1.csv"
df = pd.read_csv(src_file, dtype=str).fillna("")

# 2. Shopify 商品模板字段（可根据你的模板实际字段调整）
shopify_columns = [
    "Handle",
    "Title",
    "Body (HTML)",
    "Vendor",
    "Type",
    "Tags",
    "Published",
    "Option1 Name",
    "Option1 Value",
    "Option2 Name",
    "Option2 Value",
    "Option3 Name",
    "Option3 Value",
    "Variant SKU",
    "Variant Grams",
    "Variant Inventory Tracker",
    "Variant Inventory Qty",
    "Variant Inventory Policy",
    "Variant Fulfillment Service",
    "Variant Price",
    "Variant Compare At Price",
    "Variant Requires Shipping",
    "Variant Taxable",
    "Variant Barcode",
    "Image Src",
    "Image Position",
    "Image Alt Text",
    "Gift Card",
    "SEO Title",
    "SEO Description",
    "Google Shopping / Google Product Category",
    "Google Shopping / Gender",
    "Google Shopping / Age Group",
    "Google Shopping / MPN",
    "Google Shopping / AdWords Grouping",
    "Google Shopping / AdWords Labels",
    "Google Shopping / Condition",
    "Google Shopping / Custom Product",
    "Google Shopping / Custom Label 0",
    "Google Shopping / Custom Label 1",
    "Google Shopping / Custom Label 2",
    "Google Shopping / Custom Label 3",
    "Google Shopping / Custom Label 4",
    "Variant Image",
    "Variant Weight Unit",
    "Variant Tax Code",
    "Cost per item",
    "Status",
]


# 3. 转换函数
def telescope_to_shopify_rows(row):
    # 处理图片
    images = [img.strip('"') for img in row["Images"].split('">"') if img.strip('"')]
    if not images:
        images = [""]
    # 处理变体
    options = []
    if row["Attribute 1 name"] and row["Attribute 1 value(s)"]:
        opt_name = row["Attribute 1 name"]
        for opt_val in row["Attribute 1 value(s)"].split("|"):
            options.append((opt_name, opt_val.strip()))
    else:
        options.append(("Title", "Default Title"))

    # 展开图片和变体
    rows = []
    for img_idx, img_url in enumerate(images, 1):
        for opt_name, opt_val in options:
            shopify_row = {
                "Handle": row["SKU"].lower().replace(" ", "-"),
                "Title": row["Name"],
                "Body (HTML)": row["Description"],
                "Vendor": "Simms",
                "Type": row["Categories"],
                "Tags": row.get("Tags", ""),
                "Published": "TRUE",
                "Option1 Name": opt_name,
                "Option1 Value": opt_val,
                "Option2 Name": "",
                "Option2 Value": "",
                "Option3 Name": "",
                "Option3 Value": "",
                "Variant SKU": row["SKU"],
                "Variant Grams": "",
                "Variant Inventory Tracker": "",
                "Variant Inventory Qty": "100",
                "Variant Inventory Policy": "deny",
                "Variant Fulfillment Service": "manual",
                "Variant Price": row["Sale price"] or row["Regular price"],
                "Variant Compare At Price": row["Regular price"],
                "Variant Requires Shipping": "TRUE",
                "Variant Taxable": "TRUE",
                "Variant Barcode": "",
                "Image Src": img_url,
                "Image Position": img_idx,
                "Image Alt Text": row["Name"],
                "Gift Card": "FALSE",
                "SEO Title": row["Name"],
                "SEO Description": "",
                "Google Shopping / Google Product Category": "",
                "Google Shopping / Gender": "",
                "Google Shopping / Age Group": "",
                "Google Shopping / MPN": "",
                "Google Shopping / AdWords Grouping": "",
                "Google Shopping / AdWords Labels": "",
                "Google Shopping / Condition": "new",
                "Google Shopping / Custom Product": "",
                "Google Shopping / Custom Label 0": "",
                "Google Shopping / Custom Label 1": "",
                "Google Shopping / Custom Label 2": "",
                "Google Shopping / Custom Label 3": "",
                "Google Shopping / Custom Label 4": "",
                "Variant Image": img_url,
                "Variant Weight Unit": "g",
                "Variant Tax Code": "",
                "Cost per item": "",
                "Status": "active",
            }
            rows.append(shopify_row)
    return rows


# 4. 生成所有行
shopify_rows = []
for _, row in df.iterrows():
    shopify_rows.extend(telescope_to_shopify_rows(row))

# 5. 写出为shopify格式csv
out_file = "p1out.csv"
with open(out_file, "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.DictWriter(f, fieldnames=shopify_columns)
    writer.writeheader()
    for row in shopify_rows:
        writer.writerow(row)

print(f"转换完成，输出文件：{out_file}")

转换完成，输出文件：p1out.csv


In [3]:
import os

In [4]:
#打印当前工作目录
import os
print(os.getcwd())

c:\repos\scripts\wp\woocommerce\woo_df\csv_dir\shopyy\telescope


In [5]:
import itertools
from itertools import zip_longest
import logging
import os
import re
from logging import debug, error, info, warning

import pandas as pd

logging.basicConfig(
    level=logging.DEBUG, format=" %(name)s - %(levelname)s - %(message)s"
)
print(os.getcwd())
# 将工作目录设置为脚本所在目录
# script_path = os.path.abspath(__file__)
# d = os.path.dirname(script_path)
# os.chdir(d)

# SOURCE = r"./simms-eu.demo.csv"
SOURCE=r"./p1.csv"
# shopify产品模板可以从:https://help.shopify.com/csv/product_template.csv 下载
TEMPLATE_SHOPIFY = r"C:\repos\scripts\wp\woocommerce\woo_df\csv_dir\shopyy\templates\shopify_product_template_empty.csv"

c:\repos\scripts\wp\woocommerce\woo_df\csv_dir\shopyy\telescope


In [6]:
TEMPLATE_SHOPIFY

'C:\\repos\\scripts\\wp\\woocommerce\\woo_df\\csv_dir\\shopyy\\templates\\shopify_product_template_empty.csv'

In [7]:
# df = pd.read_csv(SOURCE)
df = pd.read_csv(SOURCE, dtype=str).fillna("")
df

,SKU,Name,Categories,Regular price,Sale price,Attribute 1 value(s),Attribute 1 name,Images,Tags,PageUrl,Description
0,SK0000001-UK,Explore Scientific Deep Sky 16MP Astro Camera ...,Colour - (Cooled),1113.22,169.99,,,https://cdn.shopify.com/s/files/1/0596/1924/62...,Explore Scientific,https://www.horizonastronomy.ie/products/explo...,<div>\n<div>\n<div>\n<div>\n<div>\n<strong>Thi...
1,SK0000002-UK,Air Blower Bulb,Cleaning Kit,6.03,3.02,,,https://cdn.shopify.com/s/files/1/0596/1924/62...,Horizon Astronomy,https://www.horizonastronomy.ie/products/air-b...,<p>Keep your Telescope and camera optics in pr...
2,SK0000003-UK,Bresser Messier EXOS-2 GOTO German Equatorial ...,Astrophotography Payload - 10kg,1010.91,169.99,,,https://cdn.shopify.com/s/files/1/0596/1924/62...,Bresser Messier,https://www.horizonastronomy.ie/products/bress...,<p><b>The Bresser EXOS-2 GoTo German Equatoria...
3,SK0000004-UK,Bresser Messier EXOS-2 German Equatorial Mount,Astrophotography Payload - 10kg,545.97,169.99,,,https://cdn.shopify.com/s/files/1/0596/1924/62...,Bresser Messier,https://www.horizonastronomy.ie/products/bress...,<p><strong>The Bresser Messier EXOS-2 (EQ-5) i...
4,SK0000005-UK,"Bresser Messier NT-203/800 EXOS-2 GOTO - 8""","8"" - Newtonian Reflector",1629.82,169.99,,,https://cdn.shopify.com/s/files/1/0596/1924/62...,Bresser Messier,https://www.horizonastronomy.ie/products/bress...,<p><strong>For many years Bresser Newtonian te...
...,...,...,...,...,...,...,...,...,...,...,...
366,SK0000367-UK,Sharpstar 61 EDPH III (incl. x0.75 Reducer),"2.5"" - Refractor",1413.05,169.99,,,https://cdn.shopify.com/s/files/1/0596/1924/62...,Askar Sharpstar,https://www.horizonastronomy.ie/products/sharp...,<p><strong>Sharpstar 61EDPH III 61mm Triplet A...
367,SK0000368-UK,Sharpstar 61 EDPH III - Field Flattening Corre...,Flattener & Reducers,353.51,169.99,,,https://cdn.shopify.com/s/files/1/0596/1924/62...,Askar Sharpstar,https://www.horizonastronomy.ie/products/sharp...,<p><strong>Sharpstar 1.0x Full-Frame Field Fla...
368,SK0000369-UK,Askar 130PHQ Astrograph,"5"" - Refractor",3919.08,169.99,,,https://cdn.shopify.com/s/files/1/0596/1924/62...,Askar Sharpstar,https://www.horizonastronomy.ie/products/askar...,<p><strong>Askar 130PHQ Quadruplet APO Astrogr...
369,SK0000370-UK,Askar Field Flattener (x0.7 Reducer) - For PHQ...,Flattener & Reducers,572.31,169.99,,,https://cdn.shopify.com/s/files/1/0596/1924/62...,Askar Sharpstar,https://www.horizonastronomy.ie/products/askar...,"<p><strong>Askar 0.7x Reducer for 107PHQ, 130P..."


In [8]:
dft = pd.read_csv(TEMPLATE_SHOPIFY)

In [21]:
dft.columns.tolist()
all_columns = dft.columns.tolist()

In [10]:
# 已有字段的映射🎈(从df_woo格式到shopify csv格式)
exist_fields_map = {
    "SKU": "Handle",
    "Name": "Title",
    "Categories": "Product Category",
    "Regular price": "Variant Compare At Price",
    "Sale price": "Variant Price",
    "Images": "Image Src",
    # "ImagesUrl": "Image Src",
    "Description": "Body (HTML)",
}
# 需要shopify 属性值字段
attr_fields_shopify = [
    "Option1 Name",
    "Option1 Value",
    "Option2 Name",
    "Option2 Value",
    "Option3 Name",
    "Option3 Value",
]
# 批量创建新列
df[attr_fields_shopify] = ""

In [11]:
df

,SKU,Name,Categories,Regular price,Sale price,Attribute 1 value(s),Attribute 1 name,Images,Tags,PageUrl,Description,Option1 Name,Option1 Value,Option2 Name,Option2 Value,Option3 Name,Option3 Value
0,SK0000001-UK,Explore Scientific Deep Sky 16MP Astro Camera ...,Colour - (Cooled),1113.22,169.99,,,https://cdn.shopify.com/s/files/1/0596/1924/62...,Explore Scientific,https://www.horizonastronomy.ie/products/explo...,<div>\n<div>\n<div>\n<div>\n<div>\n<strong>Thi...,,,,,,
1,SK0000002-UK,Air Blower Bulb,Cleaning Kit,6.03,3.02,,,https://cdn.shopify.com/s/files/1/0596/1924/62...,Horizon Astronomy,https://www.horizonastronomy.ie/products/air-b...,<p>Keep your Telescope and camera optics in pr...,,,,,,
2,SK0000003-UK,Bresser Messier EXOS-2 GOTO German Equatorial ...,Astrophotography Payload - 10kg,1010.91,169.99,,,https://cdn.shopify.com/s/files/1/0596/1924/62...,Bresser Messier,https://www.horizonastronomy.ie/products/bress...,<p><b>The Bresser EXOS-2 GoTo German Equatoria...,,,,,,
3,SK0000004-UK,Bresser Messier EXOS-2 German Equatorial Mount,Astrophotography Payload - 10kg,545.97,169.99,,,https://cdn.shopify.com/s/files/1/0596/1924/62...,Bresser Messier,https://www.horizonastronomy.ie/products/bress...,<p><strong>The Bresser Messier EXOS-2 (EQ-5) i...,,,,,,
4,SK0000005-UK,"Bresser Messier NT-203/800 EXOS-2 GOTO - 8""","8"" - Newtonian Reflector",1629.82,169.99,,,https://cdn.shopify.com/s/files/1/0596/1924/62...,Bresser Messier,https://www.horizonastronomy.ie/products/bress...,<p><strong>For many years Bresser Newtonian te...,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,SK0000367-UK,Sharpstar 61 EDPH III (incl. x0.75 Reducer),"2.5"" - Refractor",1413.05,169.99,,,https://cdn.shopify.com/s/files/1/0596/1924/62...,Askar Sharpstar,https://www.horizonastronomy.ie/products/sharp...,<p><strong>Sharpstar 61EDPH III 61mm Triplet A...,,,,,,
367,SK0000368-UK,Sharpstar 61 EDPH III - Field Flattening Corre...,Flattener & Reducers,353.51,169.99,,,https://cdn.shopify.com/s/files/1/0596/1924/62...,Askar Sharpstar,https://www.horizonastronomy.ie/products/sharp...,<p><strong>Sharpstar 1.0x Full-Frame Field Fla...,,,,,,
368,SK0000369-UK,Askar 130PHQ Astrograph,"5"" - Refractor",3919.08,169.99,,,https://cdn.shopify.com/s/files/1/0596/1924/62...,Askar Sharpstar,https://www.horizonastronomy.ie/products/askar...,<p><strong>Askar 130PHQ Quadruplet APO Astrogr...,,,,,,
369,SK0000370-UK,Askar Field Flattener (x0.7 Reducer) - For PHQ...,Flattener & Reducers,572.31,169.99,,,https://cdn.shopify.com/s/files/1/0596/1924/62...,Askar Sharpstar,https://www.horizonastronomy.ie/products/askar...,"<p><strong>Askar 0.7x Reducer for 107PHQ, 130P...",,,,,,


In [12]:
def replace_separators(s):
    """替换字符串中的分隔符
    将`|`或则`-`替换为`>`
    """
    pattern = r"/|-"
    res = re.sub(pattern, ">", s)
    return res

In [13]:
def parse_attrs(s):
    """解析属性值

    Examples:

    parse_attrs('Color#Watermelon|Neptune~Size#XS|S|M')

    root - DEBUG - ['Color', 'Size']
    root - DEBUG - [['Watermelon', 'Neptune'], ['XS', 'S', 'M']]
    (['Color', 'Size'],
        [('Watermelon', 'XS'),
        ('Watermelon', 'S'),
        ('Watermelon', 'M'),
        ('Neptune', 'XS'),
        ('Neptune', 'S'),
        ('Neptune', 'M')])

    """
    if s:
        attrs_lines = s.split("~")
        # attr_names=[name for ]
        matrix = [group.split("#") for group in attrs_lines]
        attr_names = [pair[0].strip() for pair in matrix]
        # 例如:['Color', 'Size']

        # for i, name in enumerate(attr_names):
        #     df.loc[i, "Option{i} Name"] = name
        debug(attr_names)
        attr_values = [pair[1].strip() for pair in matrix]
        # debug(attr_values)
        attr_values = [value.split("|") for value in attr_values]
        debug(attr_values)
        # 将属性值排列组合展开

        combinations = list(itertools.product(*attr_values))

        # for name, value in zip(attr_names, result):
        #     df
        return attr_names, combinations
    else:
        return ([], [])



In [14]:
img=r'https://cdn.shopify.com/s/files/1/0596/1924/6275/products/TSOpticsTelescopeDewShield_3.jpg?v=1631876333>https://cdn.shopify.com/s/files/1/0596/1924/6275/products/TSOpticsTelescopeDewShield_2.jpg?v=1631876333>https://cdn.shopify.com/s/files/1/0596/1924/6275/products/TSOpticsTelescopeDewShield_1.jpg?v=1631876333'

In [15]:
def parse_images(imgs):
    """解析图片"""
    if imgs:
        # imgs=imgs.replace('"',"")
        # res = re.findall(r'"(.*?)"', imgs)
        res = imgs.split(">")
        debug("images: %s", res)
        return res
parse_images(img)

 root - DEBUG - images: ['https://cdn.shopify.com/s/files/1/0596/1924/6275/products/TSOpticsTelescopeDewShield_3.jpg?v=1631876333', 'https://cdn.shopify.com/s/files/1/0596/1924/6275/products/TSOpticsTelescopeDewShield_2.jpg?v=1631876333', 'https://cdn.shopify.com/s/files/1/0596/1924/6275/products/TSOpticsTelescopeDewShield_1.jpg?v=1631876333']


['https://cdn.shopify.com/s/files/1/0596/1924/6275/products/TSOpticsTelescopeDewShield_3.jpg?v=1631876333',
 'https://cdn.shopify.com/s/files/1/0596/1924/6275/products/TSOpticsTelescopeDewShield_2.jpg?v=1631876333',
 'https://cdn.shopify.com/s/files/1/0596/1924/6275/products/TSOpticsTelescopeDewShield_1.jpg?v=1631876333']

In [16]:
dft.columns.tolist()

['Handle',
 'Title',
 'Body (HTML)',
 'Vendor',
 'Product Category',
 'Type',
 'Tags',
 'Published',
 'Option1 Name',
 'Option1 Value',
 'Option2 Name',
 'Option2 Value',
 'Option3 Name',
 'Option3 Value',
 'Variant SKU',
 'Variant Grams',
 'Variant Inventory Tracker',
 'Variant Inventory Qty',
 'Variant Inventory Policy',
 'Variant Fulfillment Service',
 'Variant Price',
 'Variant Compare At Price',
 'Variant Requires Shipping',
 'Variant Taxable',
 'Variant Barcode',
 'Image Src',
 'Image Position',
 'Image Alt Text',
 'Gift Card',
 'SEO Title',
 'SEO Description',
 'Google Shopping / Google Product Category',
 'Google Shopping / Gender',
 'Google Shopping / Age Group',
 'Google Shopping / MPN',
 'Google Shopping / Condition',
 'Google Shopping / Custom Product',
 'Variant Image',
 'Variant Weight Unit',
 'Variant Tax Code',
 'Cost per item',
 'Included / United States',
 'Price / United States',
 'Compare At Price / United States',
 'Included / International',
 'Price / Internationa

In [17]:


# 已有字段的映射🎈(从df_woo格式到shopify csv格式)
exist_fields_map = {
    "SKU": "Handle",
    "Name": "Title",
    "Categories": "Product Category",
    "Regular price": "Variant Compare At Price",
    "Sale price": "Variant Price",
    "Images": "Image Src",
    # "ImagesUrl": "Image Src",
    "Description": "Body (HTML)",
}
# 需要shopify 属性值字段
attr_fields_shopify = [
    "Option1 Name",
    "Option1 Value",
    "Option2 Name",
    "Option2 Value",
    "Option3 Name",
    "Option3 Value",
]
# 批量创建新列
df[attr_fields_shopify] = ""


# df.info()
def replace_separators(s):
    """替换字符串中的分隔符
    将`|`或则`-`替换为`>`
    """
    pattern = r"/|-"
    res = re.sub(pattern, ">", s)
    return res


##
# 分类
# df['Categories'].str.replace('/|-','>' )
s_cat = df["Categories"].apply(replace_separators)
df["Categories"] = s_cat


##
# 价格
# df["Sale price"] = df["Regular price"] * 0.2
s_sale_price = df["Sale price"]
##
# 属性值
attrs = df["Attribute 1 value(s)"]
# attr_lines = [attr for attr in attrs]


def parse_attrs(s: str):
    """解析属性值

    Examples:

    parse_attrs('Color#Watermelon|Neptune~Size#XS|S|M')

    root - DEBUG - ['Color', 'Size']
    root - DEBUG - [['Watermelon', 'Neptune'], ['XS', 'S', 'M']]
    (['Color', 'Size'],
        [('Watermelon', 'XS'),
        ('Watermelon', 'S'),
        ('Watermelon', 'M'),
        ('Neptune', 'XS'),
        ('Neptune', 'S'),
        ('Neptune', 'M')])

    """
    if s:
        attrs_lines = s.split("~")
        # attr_names=[name for ]
        matrix = [group.split("#") for group in attrs_lines]
        attr_names = [pair[0].strip() for pair in matrix]
        # 例如:['Color', 'Size']

        # for i, name in enumerate(attr_names):
        #     df.loc[i, "Option{i} Name"] = name
        debug(attr_names)
        attr_values = [pair[1].strip() for pair in matrix]
        # debug(attr_values)
        attr_values = [value.split("|") for value in attr_values]
        debug(attr_values)
        # 将属性值排列组合展开

        combinations = list(itertools.product(*attr_values))

        # for name, value in zip(attr_names, result):
        #     df
        return attr_names, combinations
    else:
        return ([], [])

In [18]:

def parse_images(imgs):
    """解析图片
    将多图url构成的字符串根据合适的规则分割或提取出一个个图片url字符串构成的列表
    """
    if imgs:
        # imgs=imgs.replace('"',"")
        # res = re.findall(r'"(.*?)"', imgs)
        res = imgs.split(">")
        debug("images: %s", res)
        return res

# 应用解析函数并展开
rows = []
for idx, row in df.iterrows():
    debug("processing row: %s", idx)
    attr = row["Attribute 1 value(s)"]
    print(f"attr: {attr}")
    # continue
    names, values = parse_attrs(attr or "")
    images = parse_images(row["Images"]) or []
    z = zip_longest(values, images, fillvalue="")
    for expand_line_idx, (value_group, img) in enumerate(z, start=1):
        # for i, value_group in enumerate(iterable=values, start=1):
        # 每个属性选项组占用一行(生成一个字典)
        d = {}
        debug(
            "expand_line_idx: %s, value_group: %s, img: %s",
            expand_line_idx,
            value_group,
            img,
        )
        # add_attr_name=True
        # 遍历names构造字典(字典中k:v数量取决于names的长度)
        # 商品首行
        if expand_line_idx == 1:
            d["Title"] = row["Name"]
            # d["Product Category"] = row["Categories"]
            d["Type"] = row["Categories"]
            d["Body (HTML)"] = row["Description"]
            # d["Image Src"] = row["Images"]
            # d["Image Src"] = img
            # d["Image Position"] = "1"
            d["Published"] = "TRUE"
        # 同款商品的每一行都要有的
        d["Handle"] = row["SKU"]
        # d["Variant Compare At Price"] = round(row["Regular price"], 2)
        d["Variant Compare At Price"] = round(float(row["Regular price"]), 2)
        d["Variant Price"] = round(float(row["Sale price"]), 2)
        # 多图情况
        d["Image Src"] = img
        d["Image Position"] = str(expand_line_idx)
        # 在当前行(attr_line_order行内)填充属性相关字段(option1~option3)
        options = zip(names, value_group)
        for option_order, (name, value) in enumerate(options, start=1):
            # if add_attr_name:
            debug(
                "attr_line_order: %s, option_order(in line): %s, name: %s,value: %s",
                expand_line_idx,
                option_order,
                name,  # size,color等
                value,
            )
            if expand_line_idx == 1:
                # 每组数据(一款产品占有一组数据行)的首行才添加属性名称,例如:size,color等
                # debug("Head line of this product[%s],")
                d[f"Option{option_order} Name"] = name
            else:
                d[f"Option{option_order} Name"] = ""
            d[f"Option{option_order} Value"] = value_group[option_order - 1]
        debug("check row(d): %s", d)
        rows.append(d)


 root - DEBUG - processing row: 0
 root - DEBUG - images: ['https://cdn.shopify.com/s/files/1/0596/1924/6275/products/ExploreScientific16MPAstroCamera_6daccaf7-93fe-4eef-8dbc-9c7f6eca6d76.jpg?v=1630108262', 'https://cdn.shopify.com/s/files/1/0596/1924/6275/products/ExploreScientific16MPAstroCamera_2.jpg?v=1629832454', 'https://cdn.shopify.com/s/files/1/0596/1924/6275/products/ExploreScientific16MPAstroCamera_3.jpg?v=1629832454', 'https://cdn.shopify.com/s/files/1/0596/1924/6275/products/ExploreScientific16MPAstroCamera.jpg?v=1629832454', 'https://cdn.shopify.com/s/files/1/0596/1924/6275/products/ExploreScientific16MPAstroCamera_1.jpg?v=1629832454', 'https://cdn.shopify.com/s/files/1/0596/1924/6275/files/M31Andromeda8hrs45mins-TS61EDPHII-ES16MP.jpg?v=1701561128', 'https://cdn.shopify.com/s/files/1/0596/1924/6275/files/M45Pleiades-2hrs30Mins-TS94ES16MPCamera.jpg?v=1701561128']
 root - DEBUG - expand_line_idx: 1, value_group: , img: https://cdn.shopify.com/s/files/1/0596/1924/6275/product

attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 


 root - DEBUG - expand_line_idx: 1, value_group: , img: https://cdn.shopify.com/s/files/1/0596/1924/6275/products/ExploreScientificAstroCam7MP_1.jpg?v=1632229719
 root - DEBUG - check row(d): {'Title': 'Explore Scientific Deep Sky 7.1MP Astro Camera - Colour', 'Type': 'Colour > (Cooled)', 'Body (HTML)': '<p><strong>This high resolution deep sky camera incorporates a large \'Sony Exmor IMX428 CMOS\' sensor and active cooling and is ideal for telescopes with short and moderate focal lengths.</strong><br>The Explore Scientific 7.1MP Astro camera comes in a modern complete package for the comfortable imaging of a wide range of night sky targets. This easy-to-use camera is ideal for capturing galaxies, star clusters, the moon and even planets with an extremely high sensor sensitivity and very low sensor noise. The high sensitivity enables very short exposure times and the freezing of the air turbulence (seeing) in planetary images. Thanks to the fast USB 3.0 computer interface, all image da

attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 


 root - DEBUG - processing row: 46
 root - DEBUG - images: ['https://cdn.shopify.com/s/files/1/0596/1924/6275/products/BresserCameraTripodTP-100_1.jpg?v=1631774383', 'https://cdn.shopify.com/s/files/1/0596/1924/6275/products/BresserCameraTripodTP-100_2.jpg?v=1631774383', 'https://cdn.shopify.com/s/files/1/0596/1924/6275/products/BresserCameraTripodTP-100_3.jpg?v=1631774383']
 root - DEBUG - expand_line_idx: 1, value_group: , img: https://cdn.shopify.com/s/files/1/0596/1924/6275/products/BresserCameraTripodTP-100_1.jpg?v=1631774383
 root - DEBUG - check row(d): {'Title': 'Bresser Camera/Binocular Tripod TP-100 DX (with carry bag)', 'Type': 'Binocular > Camera Tripod', 'Body (HTML)': 'Then the height-adjustable TP-100 DX aluminium tripod from Bresser is an extremely stable tripod base which can be used\xa0with the Bresser Star Tracker unit or any camera/binoculars with a 1/4 inch photo thread screw.<br>For taking astronomical photos we recommend using the Bresser Micro Motion Head MH-100

...

attr: 
attr: 
attr: 
attr: Please Choose Focuser Position#Focuser Position No.1|Focuser Position No.2|Focuser Position No.3
attr: Please Choose Focuser Position#Focuser Position No.1|Focuser Position No.2|Focuser Position No.3
attr: 
attr: 
attr: 
attr: 
attr: 
attr: Choose Telescope Version#SCT & EdgeHD 6"/8"/ 9.25" (RASA 8")|SCT & EdgeHD 11" (RASA 11" V2)|SCT & EdgeHD 14" (RASA 14")
attr: 
attr: 
attr: 
attr: 


 root - DEBUG - processing row: 342
 root - DEBUG - ['Size']
 root - DEBUG - [['6.5mm']]
 root - DEBUG - images: ['https://cdn.shopify.com/s/files/1/0596/1924/6275/files/Bresser6-5mmPlosslEyepiece.jpg?v=1751632380']
 root - DEBUG - expand_line_idx: 1, value_group: ('6.5mm',), img: https://cdn.shopify.com/s/files/1/0596/1924/6275/files/Bresser6-5mmPlosslEyepiece.jpg?v=1751632380
 root - DEBUG - attr_line_order: 1, option_order(in line): 1, name: Size,value: 6.5mm
 root - DEBUG - check row(d): {'Title': 'Bresser 6.5mm Super Plössl Eyepiece - 1.25"', 'Type': 'Eyepiece > 1.25"', 'Body (HTML)': '<p>All Bresser Plössl eyepieces provide sharp and high-contrast images because of the 4-lens design. All lenses have fully-multicoated surfaces which helps reduce reflections efficiently. Every eyepiece is equipped with a foldable rubber eyecup to avoid straylight from the sides and the inner eyepiece shaft is carefully blackened as well. The housing and eyepiece shaft are completely made from metal

attr: Size#6.5mm
attr: 
attr: 
attr: Choose Telescope Size #5" Schmidt Cassegrain|6" Schmidt Cassegrain|8" Schmidt Cassegrain|9.25" Schmidt Cassegrain|11" Schmidt Cassegrain|14" Schmidt Cassegrain
attr: 
attr: Please Select Telescope#8" EdgeHD|9.25" EdgeHD|11" EdgeHD|14" EdgeHD
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 


 root - DEBUG - check row(d): {'Handle': 'SK0000360-UK', 'Variant Compare At Price': 5316.93, 'Variant Price': 169.99, 'Image Src': 'https://cdn.shopify.com/s/files/1/0596/1924/6275/files/Celestron-Origin-Home-Intelligent-Observatory-Smart-Telescope_13.webp?v=1752687644', 'Image Position': '13'}
 root - DEBUG - expand_line_idx: 14, value_group: , img: https://cdn.shopify.com/s/files/1/0596/1924/6275/files/Celestron-Origin-Home-Intelligent-Observatory-Smart-Telescope_14.webp?v=1752687644
 root - DEBUG - check row(d): {'Handle': 'SK0000360-UK', 'Variant Compare At Price': 5316.93, 'Variant Price': 169.99, 'Image Src': 'https://cdn.shopify.com/s/files/1/0596/1924/6275/files/Celestron-Origin-Home-Intelligent-Observatory-Smart-Telescope_14.webp?v=1752687644', 'Image Position': '14'}
 root - DEBUG - expand_line_idx: 15, value_group: , img: https://cdn.shopify.com/s/files/1/0596/1924/6275/files/Celestron-Origin-Home-Intelligent-Observatory-Smart-Telescope_15.webp?v=1752687644
 root - DEBUG - 

attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 
attr: 


In [22]:
variants = pd.DataFrame(rows, columns=all_columns)

In [23]:
variants


,Handle,Title,Body (HTML),Vendor,Product Category,Type,Tags,Published,Option1 Name,Option1 Value,...,Variant Weight Unit,Variant Tax Code,Cost per item,Included / United States,Price / United States,Compare At Price / United States,Included / International,Price / International,Compare At Price / International,Status
0,SK0000001-UK,Explore Scientific Deep Sky 16MP Astro Camera ...,<div>\n<div>\n<div>\n<div>\n<div>\n<strong>Thi...,NaN,NaN,Colour > (Cooled),NaN,TRUE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SK0000001-UK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SK0000001-UK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SK0000001-UK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SK0000001-UK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030,SK0000371-UK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2031,SK0000371-UK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2032,SK0000371-UK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2033,SK0000371-UK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
variants[["Image Src", "Image Position"]]

,Image Src,Image Position
0,https://cdn.shopify.com/s/files/1/0596/1924/62...,1
1,https://cdn.shopify.com/s/files/1/0596/1924/62...,2
2,https://cdn.shopify.com/s/files/1/0596/1924/62...,3
3,https://cdn.shopify.com/s/files/1/0596/1924/62...,4
4,https://cdn.shopify.com/s/files/1/0596/1924/62...,5
...,...,...
2030,https://cdn.shopify.com/s/files/1/0596/1924/62...,5
2031,https://cdn.shopify.com/s/files/1/0596/1924/62...,6
2032,https://cdn.shopify.com/s/files/1/0596/1924/62...,7
2033,https://cdn.shopify.com/s/files/1/0596/1924/62...,8


In [27]:
variants.to_csv("changed_demox.csv", index=False)
